In [6]:
%load_ext autoreload
%autoreload 2

# Week 6 - Systematically Improving Your Rag Application

> If you haven't already, please run the [1. Evaluate Tools](1. Evaluate Tools.ipynb) notebook to understand how to evaluate your application's tool calling capabilities. That will help you understand what we're doing here in this notebook.

## Generating Synthetic Data

We want to evaluate the recall of our application's tool calling capabilities. To do so, we'll generate a synthetic dataset of queries using a small subset of Raycast's extensions. We'll do so in 3 steps

1. **Extracting Commands** : We'll first start by extracting the commands from the `package.json` file from the extensions we'd like to generate synthetic queries for
2. **Generating Queries** : We'll then generate a set of test queries for each of these commands
3. **Evaluating Recall** : We'll then evaluate the precision and recall of our application's tool calling capabilities for each of these queries

We'll do so by extracting the commands from the `package.json` files for the extensions we'd like to run our analysis on. We'll then generate a set of test queries for each of these commands and evaluate the recall for each command and query.


In [1]:
VOICE_COMMAND_EXTENSIONS = [
    "google-search",
    "jira-search",
    "notion",
    "apple-notes",
    "obsidian",
    "calendar",
    "wikipedia",
    "youtube",
    "amazon-search",
    "reddit-search",
    "spotify-player",
    "1password",
]

In [3]:
import os
import requests
import json

def ensure_directory(directory: str):
    """Ensure the given directory exists."""
    if not os.path.exists(directory):
        os.makedirs(directory)


def fetch_package_json(extension_name: str) -> str:
    """Generate the URL to fetch the package.json of the extension."""
    return f"https://raw.githubusercontent.com/raycast/extensions/main/extensions/{extension_name}/package.json"


def download_package_json(extension_name: str, output_dir: str) -> bool:
    """Download the package.json for a specific extension."""
    url = fetch_package_json(extension_name)
    response = requests.get(url)
    if response.status_code == 200:
        with open(f"{output_dir}/{extension_name}.json", "w") as file:
            file.write(response.text)
        print(f"Downloaded package.json for {extension_name}")
        return True
    else:
        print(f"Failed to download package.json for {extension_name}")
        return False

def extract_commands(extension_name: str, input_dir: str) -> list:
    """Extract commands from a package.json file."""
    commands = []
    file_path = f"{input_dir}/{extension_name}.json"
    try:
        with open(file_path, "r") as file:
            package_data = json.load(file)
            for command in package_data.get("commands", []):
                commands.append(
                    {
                        "name": command["name"],
                        "description": command.get(
                            "description", "No description provided"
                        ),
                        "extension": extension_name,
                        "plugin_description": package_data.get(
                            "description", "No description provided"
                        ),
                    }
                )
    except FileNotFoundError:
        print(f"File not found for {extension_name}")
    except json.JSONDecodeError:
        print(f"Failed to decode JSON for {extension_name}")
    return commands


We can then extract the commands from the `package.json` files and download them into a single `json` file that we can use to generate our synthetic dataset.

In [5]:
output_dir = "packages"
ensure_directory(output_dir)

# Step 1: Download package.json files
for extension in VOICE_COMMAND_EXTENSIONS:
    download_package_json(extension, output_dir)

# Step 2: Extract and display commands
all_commands = []
for extension in VOICE_COMMAND_EXTENSIONS:
    commands = extract_commands(extension, output_dir)
    all_commands.extend(commands)

# Save extracted commands to a JSON file
with open("extracted_commands.json", "w") as file:
    json.dump(all_commands, file, indent=2)

# Print extracted commands to console
for command in all_commands:
    print(f"[{command['extension']}] {command['name']}: {command['description']}")

Downloaded package.json for google-search
Downloaded package.json for jira-search
Downloaded package.json for notion
Downloaded package.json for apple-notes
Downloaded package.json for obsidian
Downloaded package.json for calendar
Downloaded package.json for wikipedia
Downloaded package.json for youtube
Downloaded package.json for amazon-search
Downloaded package.json for reddit-search
Downloaded package.json for spotify-player
Downloaded package.json for 1password
[google-search] index: Google search with autosuggestions
[google-search] searchSelectText: Google search with selected text
[jira-search] issue: Search Jira issues by text, type and project
[jira-search] board: Search Jira boards by title
[jira-search] filter: Search Jira filters by title
[jira-search] project: Search Jira projects by title
[jira-search] image: Delete cached Jira images and reload them on next search
[notion] create-database-page: Create a page in a Notion database.
[notion] search-page: Search for Notion d